In [1]:
import pandas as pd

from Code.UtilityFunctions.run_query import run_query

In [2]:
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_colwidth', 500)

### Create category file

In [ ]:
query = """
SELECT DISTINCT ?s
WHERE {
    ?s ?p yelpont:YelpCategory .
}
"""

categories = run_query(query, as_dataframe=True)
categories

In [8]:
# APPENDS to the .ttl file
with open(file="yelp_categories.ttl", mode="a") as file:
    file.write("@prefix yelpont: <https://purl.archive.org/purl/yelp/ontology#> \n@prefix yelpcat: <https://purl.archive.org/purl/yelp/business_categories#> \n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> \n")
    for category in categories['s.value']:
        file.write(f'\nyelpcat:{category} a yelpont:YelpCategory ; \n    rdfs:label "{category} is a category in Yelp used to describe a business." . \n')

### Template for Yelp ontology file

In [ ]:
query = """
SELECT DISTINCT ?o
WHERE {
    ?s ?p ?o .
    FILTER regex(?o, "^https://purl")
}
"""

predicates = run_query(query, as_dataframe=True)

In [ ]:
# APPENDS to the .ttl file
with open(file="yelp_ontology.ttl", mode="a") as file:
    for predicate in predicates['p.value']:
        file.write(f'\n <{predicate}> \n a owl:ToDO ; \n rdfs:comment "Specifies something ..."@en ; \n rdfs:domain yelpont:X ; \n rdfs:label "{predicate}"@en ; \n rdfs:range yelpont:X . \n')

In [ ]:
query = """
SELECT DISTINCT ?o
WHERE {
    ?s ?p ?o .
    FILTER regex(?o, "^https://purl")
}
"""

predicates = run_query(query, as_dataframe=True)

### Number of predicates in Yelp ontology

In [ ]:
query = """
SELECT DISTINCT(?p)
FROM <http://www.yelpkg.com/yelp_kg>
WHERE {
    ?s ?p ?o .
    FILTER regex(?p, "^https://purl.archive.org/purl/yelp/yelp_ontology")
}
"""

run_query(query, as_dataframe=True)